In [1]:
# pip install Selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas
import matplotlib.pyplot as plt

In [3]:
# 台北市各地區代碼
item={1:'Zhongshan',2:'Datong',3:'Zhongshan',4:'Songshan',5:'Da’an',6:'Wanhua',
      7:'Xinyi',8:'Shilin',9:'Beitou',10:'Neihu',11:'Nangang',12:'Wenshan'}

In [5]:
# 獲取台北市各地區新住宅頁數
pages={}
for i in range(12):
    url=f'https://newhouse.591.com.tw/housing-list.html?rid=1&sid={i+1}'

    s=Service(r'/Users/chenyaoxuan/Desktop/tools/chromedriver/chromedriver')
    chrome=webdriver.Chrome(service=s)
    chrome.get(url)

    time.sleep(5)
    soup=BeautifulSoup(chrome.page_source,'lxml')
    page=soup.find('div',class_="page-limit page-housing").find_all('a',class_="pageNum-form")[-1].text
    pages[i]=eval(page)
    
    chrome.quit()
pages

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 100
Current browser version is 102.0.5005.61 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x00000001012dbc34 chromedriver + 4406324
1   chromedriver                        0x0000000101275290 chromedriver + 3986064
2   chromedriver                        0x0000000100ed684c chromedriver + 190540
3   chromedriver                        0x0000000100ef8200 chromedriver + 328192
4   chromedriver                        0x0000000100ef4c48 chromedriver + 314440
5   chromedriver                        0x0000000100ef1c30 chromedriver + 302128
6   chromedriver                        0x0000000100f22db8 chromedriver + 503224
7   chromedriver                        0x0000000100efa864 chromedriver + 338020
8   chromedriver                        0x00000001012a1320 chromedriver + 4166432
9   chromedriver 

In [ ]:
# 抓取台北市各地區所有新住宅資料
# 共計12區
myData=[]
for j in range(12):
    totalPrice=0
    count=0
#     各地區總頁數
    for k in range(pages[j]):
        url=f'https://newhouse.591.com.tw/housing-list.html?rid=1&sid={j+1}&page={k+1}'

        s=Service(r'/Users/chenyaoxuan/Desktop/chromedriver')
        chrome=webdriver.Chrome(service=s)
        chrome.get(url)

        time.sleep(5)
        soup=BeautifulSoup(chrome.page_source,'lxml')
        datas=soup.find('div',id="vue-list").find('ul',class_="clearfix").find_all('li')
#         該頁所有新住宅價位
        for data in datas:
            try:
#                 過濾資料
                if data.find('p',class_="averPrice").find('span',class_="unite").text[2]=='坪':
                    price=eval(data.find('p',class_="averPrice").find('em',class_="stonefont").text[0:3].strip('.~'))
                    totalPrice+=price
                    count+=1
#                     print(price,count)
            except:
                pass
            finally:
                chrome.quit()
            
    avgPrice=round(totalPrice/count,2)
#     將資料寫入記憶體
    myData.append([item[j+1],avgPrice])
    print(f'{item[j+1]}平均房價為:{avgPrice}萬/坪')

In [ ]:
# 將記憶體資料匯成csv檔
pandas.DataFrame(myData).to_csv('taipeiHousePrice.csv',encoding='utf-8-sig')

In [ ]:
# 將記憶體資料視覺化
plt.figure(figsize=(20,10))
x=[]
y=[]
for xData in myData:
    x.append(xData[0])
for yData in myData:
    y.append(yData[1])
plt.title('taipeiHousePrice',fontsize=40)
plt.xlabel('place',fontsize=40)
plt.xticks(fontsize=20,rotation=45)
plt.ylabel('price',fontsize=40)
plt.yticks(fontsize=20)
plt.bar(x,y)
plt.show()
# 將視覺化資料匯成png檔
plt.savefig('taipeiHousePrice.png')

# test
url='https://newhouse.591.com.tw/housing-list.html?rid=1&sid=1&page=1'

s=Service(r'/Users/chenyaoxuan/Desktop/chromedriver')
chrome=webdriver.Chrome(service=s)
chrome.get(url)

time.sleep(5)
soup=BeautifulSoup(chrome.page_source,'lxml')
datas=soup.find('div',id="vue-list").find('ul',class_="clearfix").find_all('li')
datas